In [1]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd

C:\Users\User\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
slp_score_data = pd.read_csv("../EDA/slp_score_data.csv")
sleep_data = pd.read_csv("../EDA/sleep_data.csv")
wellness_data = pd.read_csv("../EDA/wellness_data.csv")

processed_summary= pd.read_csv("../EDA/sleep_processed_summary.csv")
processed_data= pd.read_csv("../EDA/sleep_processed_data.csv")
processed_shortdata= pd.read_csv("../EDA/sleep_processed_shortdata.csv")

In [3]:
wellness_data

,effective_time_frame,fatigue,mood,readiness,sleep_duration_h,sleep_quality,soreness,soreness_area,stress,participant_id,date
0,2019-11-01 08:31:40.751000+00:00,2,3,5,6,3,2,[12921003],3,p01,2019-11-01
1,2019-11-02 10:00:01.229000+00:00,2,3,6,6,3,2,[12921003],3,p01,2019-11-02
2,2019-11-03 14:28:03.263000+00:00,3,3,8,6,3,3,[],3,p01,2019-11-03
3,2019-11-04 07:05:28.429000+00:00,3,3,8,6,3,3,[],3,p01,2019-11-04
4,2019-11-05 06:13:35.998000+00:00,3,3,8,5,3,3,[],3,p01,2019-11-05
...,...,...,...,...,...,...,...,...,...,...,...
1742,2020-03-08 11:44:44.398000+00:00,3,3,4,6,2,3,[],3,p16,2020-03-08
1743,2020-03-11 11:33:15.168000+00:00,2,3,4,6,2,3,[],2,p16,2020-03-11
1744,2020-03-15 08:33:15.985000+00:00,4,4,4,10,4,4,[],4,p16,2020-03-15
1745,2020-03-17 09:54:27.989000+00:00,2,3,4,5,3,3,[],3,p16,2020-03-17


In [4]:
def synthesiser(data, id_col, participant_col, start_participant_num, num_new_participants):
    """
    Generate synthetic data for new participants using CTGAN.

    Parameters:
    - data (pd.DataFrame): Original dataset.
    - id_col (str): Column name to be treated as unique row ID (e.g., 'row_id').
    - participant_col (str): Column name representing participant IDs.
    - start_participant_num (int): Number to start naming new participants from (e.g., 17 for 'p17').
    - num_new_participants (int): How many new participants to synthesize.

    Returns:
    - pd.DataFrame: Synthetic data for the new participants.
    """
    
    # Step 0: Assign unique row ID if not already
    if id_col not in data.columns:
        data[id_col] = range(len(data))
    
    # Step 1: Setup metadata
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data)
    metadata.update_column(column_name=id_col, sdtype='id')
    metadata.set_primary_key(column_name=id_col)
    metadata.update_column(column_name=participant_col, sdtype='categorical')

    # Step 2: Fit the synthesizer
    synthesizer = CTGANSynthesizer(metadata)
    synthesizer.fit(data)

    # Step 3: Estimate rows per participant and sample synthetic data
    original_participant_count = data[participant_col].nunique()
    rows_per_participant = len(data) // original_participant_count
    synthetic_sample_size = rows_per_participant * num_new_participants
    synthetic_data = synthesizer.sample(synthetic_sample_size)

    # Step 4: Generate new participant IDs
    new_ids = [f"p{i}" for i in range(start_participant_num, start_participant_num + num_new_participants)]

    # Step 5: Build new synthetic dataset
    new_participant_data = pd.DataFrame()

    for new_id in new_ids:
        subset = synthetic_data.sample(rows_per_participant, replace=True).copy()
        subset[participant_col] = new_id
        new_participant_data = pd.concat([new_participant_data, subset], ignore_index=True)

    # Step 6: Reassign row IDs
    new_participant_data[id_col] = range(1, len(new_participant_data) + 1)

    return new_participant_data

In [5]:
synthetic_wellness = synthesiser(
    data=wellness_data,
    id_col='row_id',
    participant_col='participant_id',
    start_participant_num=17,
    num_new_participants=84
)
wellness_final = pd.concat([wellness_data, synthetic_wellness], ignore_index=True)
wellness_final.describe()

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'effective_time_frame'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,fatigue,mood,readiness,sleep_duration_h,sleep_quality,soreness,stress,row_id
count,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000
mean,2.555810,3.110337,4.675044,6.812804,3.043658,2.723746,2.649454,3984.763551
std,0.970288,1.009325,2.260581,1.332907,0.984992,0.888269,1.012589,2784.902112
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,3.000000,6.000000,2.000000,2.000000,2.000000,1363.000000
50%,3.000000,3.000000,5.000000,7.000000,3.000000,3.000000,3.000000,3705.000000
75%,3.000000,4.000000,6.000000,7.000000,4.000000,3.000000,3.000000,6430.500000
max,5.000000,5.000000,10.000000,12.000000,5.000000,5.000000,5.000000,9156.000000


In [6]:
wellness_final.to_csv("GAN_generated_data/wellness_synthesised_100_participants.csv", index=False)

In [7]:
synthetic_slp_score = synthesiser(
    data=slp_score_data,
    id_col='sleep_log_entry_id',
    participant_col='participant_id',
    start_participant_num=17,
    num_new_participants=84
)
slp_score_final = pd.concat([slp_score_data, synthetic_slp_score], ignore_index=True)
slp_score_final.describe()

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'sleep_log_entry_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,sleep_log_entry_id,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness
count,1.141200e+04,11412.000000,11412.000000,11412.000000,11412.000000,11412.000000,11412.000000,11412.000000
mean,4.098343e+09,71.449264,19.561251,18.990887,37.514108,66.824658,57.343761,0.086773
std,9.363037e+09,9.381680,2.403080,3.271264,5.675605,31.710996,7.818489,0.043431
min,1.000000e+00,35.000000,12.000000,5.000000,3.000000,0.000000,44.000000,0.015385
25%,2.853750e+03,66.000000,17.000000,18.000000,34.000000,45.000000,51.000000,0.055720
50%,5.706500e+03,72.000000,20.000000,19.000000,39.000000,66.000000,59.000000,0.079653
75%,8.559250e+03,78.000000,21.000000,21.000000,41.000000,87.000000,63.000000,0.109593
max,2.654316e+10,94.000000,25.000000,25.000000,47.000000,183.000000,76.000000,0.294766


In [8]:
slp_score_final.to_csv("GAN_generated_data/sleep_score_synthesised_100_participants.csv", index=False)

In [9]:
sleep_data = sleep_data.drop(columns=["levels"])

synthetic_overall_sleep = synthesiser(
    data=sleep_data,
    id_col='row_id',
    participant_col='participant_id',
    start_participant_num=17,
    num_new_participants=84
)
overall_sleep_final = pd.concat([sleep_data, synthetic_overall_sleep], ignore_index=True)
overall_sleep_final.describe()

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'row_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,logId,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,infoCode,row_id
count,1.290000e+04,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.00000
mean,2.556849e+10,340.549535,0.864961,343.828295,38.307519,0.504031,350.353488,95.150465,0.459380,4716.58000
std,5.925099e+08,165.347198,4.289623,156.114124,28.195574,1.505830,165.689290,3.926894,0.817274,3295.99718
min,2.447235e+10,60.000000,0.000000,39.000000,0.000000,0.000000,60.000000,0.000000,0.000000,0.00000
25%,2.508231e+10,219.000000,0.000000,265.750000,15.000000,0.000000,253.000000,93.000000,0.000000,1612.75000
50%,2.554535e+10,360.000000,0.000000,384.000000,38.000000,0.000000,385.000000,96.000000,0.000000,4386.50000
75%,2.608390e+10,457.000000,0.000000,453.000000,57.000000,0.000000,464.000000,98.000000,0.000000,7611.25000
max,2.655419e+10,953.000000,35.000000,823.000000,170.000000,33.000000,953.000000,100.000000,2.000000,10836.00000


In [10]:
overall_sleep_final.to_csv("GAN_generated_data/overall_sleep_synthesised_100_participants.csv", index=False)